UNIVERSIDADE ESTADUAL DO OESTE DO PARANÁ - UNIOESTE
PROGRAMA DE PÓS GRADUAÇÃO EM CIÊNCIA DA COMPUTAÇÃO – PPGCOMP


Estruturas de Dados e Análise de Algoritmos - EDAA



Atividade 1.2 – Algoritmos de Busca – Parte 2


##1) Descrição
A utilização de listas ligadas tem vantagens e desvantagens de acordo com o problema que está
sendo resolvido. Por vezes, a manipulação dos dados (inserção e remoção, por exemplo) é mais otimizada.
Porém, busca e ordenação, podem ser menos custosas utilizando vetores.

Esta atividade individual consiste em implementar e comparar empiricamente a eficiência dos
seguintes métodos de busca considerando estruturas de dados lineares e não lineares com valores inteiros:
* Busca sequencial padrão em listas ligadas;
* Busca em árvores binárias de busca.

## 2) Casos de Teste
Utilize os casos de teste da Atividade 1.1. Considere os mesmos
vetores gerados aleatoriamente antes da ordenação. Insira os elementos na lista ou árvore na ordem em que foram gerados

## 3) Execução
* Utilize a mesma linguagem da atividade de avaliação anterior.
* O custo de criação do arranjo e das estruturas dinâmicas devem ser desconsiderados;
* Para as árvores binárias de busca, informe a altura máxima das árvore gerada em cada cenário.
* Execute as mesmas buscas da atividade anterior, incluindo o pior caso (e repetições) e os casos médios (100 buscas), calculando a média e o desvio padrão;

## 4) Entrega
Os resultados devem ser apresentados em até 10 (dez) páginas em PDF no formato de artigos da SBC – Sociedade Brasileira de Computação disponível em [Modelos para Publicação de Artigos](https://www.sbc.org.br/documentos-da-sbc/category/169-templates-para-artigos-e-capitulos-de-livros). O artigo
deve incluir os resultados da atividade anterior, comparando os métodos de busca de acordo com as suas similaridades.

No relatório:

**Resumo/abstract**: deve conter minimamente o objetivo do trabalho e os principais resultados (máx. 10
linhas).

**Introdução**: contextualização do problema, breve descrição dos algoritmos, sequência das próximas seções
do artigo (uma página, uma página e meia).

**Materiais e métodos**: descrição do ambiente (computador, Sistema Operacional, linguagem, pacotes extras,
etc.) e cenários de testes (até uma página).

**Resultados**: além dos dados quantitativos, incluir uma discussão qualitativa dos resultados. Procure
identificar melhores e piores usos para cada solução.

**Conclusão**: retoma-se o objetivo do trabalho, a metodologia e apresenta-se os principais resultados (~meia
página).

**Referências bibliográficas**: fontes confiáveis.

## Configuração do ambiente

In [1]:
# Listar locales instalados
!locale -a

C
C.UTF-8
en_US.utf8
POSIX


In [2]:
# Instalar o locale pt_BR
!/usr/share/locales/install-language-pack pt_BR
!dpkg-reconfigure locales

Generating locales (this might take a while)...
  pt_BR.ISO-8859-1... done
Generation complete.
dpkg-trigger: error: must be called from a maintainer script (or with a --by-package option)

Type dpkg-trigger --help for help about this utility.
Generating locales (this might take a while)...
  en_US.UTF-8... done
  pt_BR.ISO-8859-1... done
Generation complete.


In [3]:
# Listar locales instalados
!locale -a

C
C.UTF-8
en_US.utf8
POSIX
pt_BR
pt_BR.iso88591


In [ ]:
# Reinicar o processo do python para enxergar o locale pt_BR
import os
os.kill(os.getpid(), 9)
# A execução do notebook será interrompida
# Será necessário disparar nova executação a partir da célula abaixo (CTRL+F10)

In [1]:
# Ignorar alerta de itens depreciados
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Definir o locale pt_BR
import locale
locale.setlocale(locale.LC_NUMERIC, 'pt_BR.ISO8859-1')

'pt_BR.ISO8859-1'

## Código

### Pacotes utilizados

In [3]:
# pacotes utilizados
import random                               # Utilizado na geração do arranjo e da chave de busca - inteiros (pseudo)aleatórios
import pandas as pd                         # Utilizado para calcular os resultados agregados das execuções

from math import sqrt                       # Utilizado no cálculo do tamanho do salto/bloco - busca_por_saltos
from collections import namedtuple          # Utilizado para armazenar os resultados da execução de cada busca realizada
from collections.abc import Sequence        # Type hint nas funções que aceitam arranjos
from timeit import default_timer as timer   # Utilizado na temporização das buscas e ordenações
from types import FunctionType              # Type hint na função executar_busca, que aceita uma função metodo (de busca) como argumento
from typing import Union                    # Type hint nas funções de busca, que retornam um int ou None
from typing import TypeVar                  # Type hint nas estruturas de dados

### Definições

In [4]:
# Definições
TAM_ARRANJO_MIN =   100_000
TAM_ARRANJO_MAX = 1_000_000
TAM_ARRANJO_INC =   100_000

EXEC_DESCARTAR_MAIOR = 2
EXEC_ORDENACAO = 3 + EXEC_DESCARTAR_MAIOR
EXEC_PIOR_CASO = 3 + EXEC_DESCARTAR_MAIOR
EXEC_BUSCAS = 100 + EXEC_DESCARTAR_MAIOR

FATOR_ESPALHAMENTO = 100

# semente de aleatoriedade fixa para assegurar reprodutibilidade
random.seed(42)     # o sentido da vida, o universo e tudo mais

### Estruturas de dados

In [5]:
# Tipo genérico para os dados armazenados nas estruturas de dados
T = TypeVar("T")

#### Árvore binária de busca

In [6]:
# Nó para árvores
class NoArvore:
  def __init__(self, dado: T, esquerda: 'NoArvore' = None, direita: 'NoArvore' = None, pai: 'NoArvore' = None, nivel: int = None, indice: int = None):
    self.dado = dado
    self.esquerda = esquerda
    self.direita = direita
    self.pai = pai
    self.nivel = nivel
    self.indice = indice

  def __str__(self):
    return f'{self.dado}'

In [7]:
# Árvore binária de busca
class Arvore:
  def __init__(self):
    self.raiz: NoArvore = None
    self.tamanho = 0
    self.altura = 0

  def inserir(self, no: NoArvore):
    self.tamanho += 1

    if self.raiz == None:
      no.nivel = 0
      no.indice = 0
      self.raiz = no
      self.altura = 1
      return

    altura = 2
    atual = self.raiz
    if no.dado <= atual.dado:
      proximo = atual.esquerda
    else:
      proximo = atual.direita

    while (proximo != None):
      atual = proximo
      proximo = proximo.esquerda if no.dado <= proximo.dado else proximo.direita
      altura += 1

    no.indice = self.tamanho - 1
    no.nivel = altura - 1
    no.pai = atual
    if no.dado <= atual.dado:
      atual.esquerda = no
    else:
      atual.direita = no

    self.altura = max(altura, self.altura)

  @classmethod
  def pior_caso(cls, arranjo_ordenado: Sequence):
    '''
    Agiliza a criação de uma árvore a partir de um arranjo ordenado (crescente)
    '''
    arvore = cls()
    arvore.tamanho = len(arranjo_ordenado)
    arvore.altura = len(arranjo_ordenado)
    posicao = 0
    arvore.raiz = NoArvore(dado=arranjo_ordenado[posicao], nivel=posicao, indice=posicao)
    ultimo = arvore.raiz
    for posicao in range(1, len(arranjo_ordenado)):
      no = NoArvore(dado=arranjo_ordenado[posicao], nivel=posicao, indice=posicao)
      no.pai = ultimo
      ultimo.direita = no
      ultimo = no
    return arvore

  def busca(self, chave: T) -> tuple[Union[NoArvore, None], int]:
    '''
    Implementação  do algoritmo de busca em árvores binárias de busca.
    Divide sucessivamente o arranjo ao meio, comparando elemento central com a chave, caso este seja:
    igual à chave    : retorna a posição
    maior que a chave: busca no segmento inferior [esquerda, meio - 1]
    menor que a chave: busca no segmento superior [meio + 1, direita ]
    Parâmetros obrigatórios:
    arranjo: sequência de elementos ordenados
    chave: chave de busca
    Parâmetros opcionais (nomeadods):
    '''
    comparacoes = 0

    atual = self.raiz
    while atual:
      comparacoes += 1
      if chave == atual.dado:
        return (atual, comparacoes)
      elif chave < atual.dado:  # deve-se buscar na subárvore esquerda
        atual = atual.esquerda
      else:
        atual = atual.direita   # deve-se buscar na subárvore direita

    return (None, comparacoes) # Não encontrou

  def percurso_largura(self):
    lista = list()
    lista.insert(0, self.raiz)
    while lista:
      atual = lista.pop()
      if atual.esquerda:
        lista.insert(0, atual.esquerda)
      if atual.direita:
        lista.insert(0, atual.direita)
      print(f'#{atual.indice}={atual.dado}')

#### Lista encadeada

In [8]:
# Nó para listas encadeadas
class NoLista:
  def __init__(self, dado: T, proximo: 'NoLista' = None, anterior: 'NoLista' = None, indice: int = None):
    self.dado = dado
    self.proximo = proximo
    self.anterior = anterior
    self.indice = indice

  def __str__(self):
    return f'{self.dado}'

In [9]:
# Lista duplamente encadeada
class Lista:
  def __init__(self):
    self.cabeca: NoLista = None
    self.calda: NoLista = None
    self.tamanho = 0

  def inserir(self, no: NoLista):
    self.tamanho += 1

    if self.cabeca == None:
      no.indice = 0
      self.cabeca = no
      self.calda = no
      return

    no.indice = self.tamanho - 1

    calda = self.calda
    no.anterior = calda
    calda.proximo = no
    self.calda = no

  def busca(self, chave: T) -> tuple[Union[NoLista, None], int]:
    '''
    Implementação do algoritmo da busca sequencial em listas.
    Percorre todo o arranjo, comparando cada elemento elemento com a chave de busca.
    Retorna a primeira ocorrência ou None caso o elemento não esteja presente no arranjo.
    Parâmetros obrigatórios:
    lista: lista encadeada de elementos
    chave: chave de busca
    Parâmetros opcionais (nomeados):
    '''
    comparacoes = 0

    atual = self.cabeca
    while atual:
      comparacoes += 1
      if atual.dado == chave:
        return (atual, comparacoes)
      atual = atual.proximo

    return (None, comparacoes) # Não encontrou

#### Armazenamento de resultados

In [10]:
# variáveis globais
resultados = list()

In [11]:
Resultado = namedtuple('Resultado', [
    'tamanho_arranjo',
    'rodada',
    'min',
    'max',
    'chave',
    'presente',
    'metodo',
    'comparacoes',
    'tempo',
    'posicao',
    'cenario',
    'profundidade'
])

In [12]:
Resultado.__doc__ += ': Armazena o resultado de uma execução de busca em arranjo'
Resultado.tamanho_arranjo.__doc__ = 'Tamanho do arranjo no qual se efetuou a busca'
Resultado.rodada.__doc__ = 'Número da rodada de testes'
Resultado.min.__doc__ = 'Menor valor no arranjo'
Resultado.max.__doc__ = 'Maior valor no arranjo'
Resultado.chave.__doc__ = 'Chave buscada'
Resultado.presente.__doc__ = 'Indica se o valor da chave está presente ou não no arranjo'
Resultado.metodo.__doc__ = 'Método de busca utilizado'
Resultado.comparacoes.__doc__ = 'Número de comparações efetuados'
Resultado.tempo.__doc__ = 'Tempo gasto na busca/ordenação'
Resultado.posicao.__doc__ = 'Posição na qual a chave de busca foi encontrada'
Resultado.cenario.__doc__ = 'Cenário da execução ordenação ou busca: aleatório ou pior caso'
Resultado.profundidade.__doc__ = 'Profundidade da estrutura de dados. Para árvores, corresponde a altura. Outras estruturas = None'

In [13]:
def registrar_resultado(resultado: Resultado):
  '''
  Salva um resultado individual na lista de resultados
  '''
  global resultados
  resultados.append(resultado)

### Geração das estruturas e chaves de busca

In [14]:
def gerar_arranjo(tamanho: int) -> Sequence:
  '''
  Gera um arranjo aleatório não ordenado com tamanho elementos e valores entre 0 e o tamanho do arranjo * o fator de espalhamento
  '''
  return random.sample(range(tamanho * FATOR_ESPALHAMENTO), tamanho)

In [15]:
# Funções acessórias para criar as estruturas de dados a partir de Sequence
def criar_lista(vetor: Sequence) -> Lista:
  lista = Lista()
  for elemento in vetor:
    lista.inserir(NoLista(elemento))
  return lista

def criar_arvore(vetor: list) -> Arvore:
  arvore = Arvore()
  for elemento in vetor:
    arvore.inserir(NoArvore(elemento))
  return arvore

In [16]:
def gerar_chave_aleatoria(arranjo: Sequence) -> int:
  '''
  Gera uma chave de busca escolhendo, com igual probabilidade, entre:
  um valor qualquer entre 0 e o tamanho do arranjo * o fator de espalhamento e
  um valor qualquer presente no arranjo
  Probabilidade da chave estar presente no arranjo = 1/fator espalhamento + 0,5
  '''
  chave_existente = random.choice(arranjo)
  return random.choice([random.randrange(len(arranjo) * FATOR_ESPALHAMENTO), chave_existente])

In [17]:
def gerar_chave_pior_caso(arranjo: Sequence) -> int:
  '''
  Dado um um arranjo, gera uma chave que corresponde ao pior caso para todos os algoritmos de busca considerados
  '''
  # Busca sequencial em listas: Percorrer toda a lista (chave último elemento ou chave inexistente)
  # Busca em árvores binárias : Percorrer o maior caminho (chave último no maior nível ou sucessora)
  chave = arranjo[-1]  # Chave de pesquisa igual ao último elemento
  return chave

### Execução das buscas

In [18]:
def executar_busca(arranjo: Union[Lista, Arvore], chave: int, **kwargs) -> tuple[Union[int, None], int]:
  '''
  Para um dado cenário, busca a chave no arranjo utilizando o método de busca do arranjo
  e registra o resultado da execução.
  '''

  tempo = timer()
  elemento, comparacoes = arranjo.busca(chave)
  posicao = elemento.indice if elemento else None
  tempo = timer() - tempo
  profundidade = estrutura.altura if isinstance(estrutura, Arvore) else None

  registrar_resultado(Resultado(
      metodo=arranjo.__class__.__name__.capitalize(),
      chave=chave,
      comparacoes=comparacoes,
      tempo=tempo,
      posicao=posicao,
      profundidade=profundidade,
      **kwargs
  ))

  return (elemento, comparacoes)

In [19]:
# Executa as buscas para os diversos tamanhos de arranjo
for tamanho_arranjo in range(TAM_ARRANJO_MIN, TAM_ARRANJO_MAX + 1, TAM_ARRANJO_INC):
  arranjo = gerar_arranjo(tamanho_arranjo)
  menor = min(arranjo)
  maior = max(arranjo)

  # Cenário: Aleatório
  lista  = criar_lista(arranjo)
  arvore = criar_arvore(arranjo)
  estruturas = (lista, arvore)
  for rodada in range(0, EXEC_BUSCAS):
    chave = gerar_chave_aleatoria(arranjo)
    kwargs = {
      'min': menor,
      'max': maior,
      'tamanho_arranjo': tamanho_arranjo,
      'presente': chave in arranjo,
      'rodada': rodada,
      'cenario': 'Aleatório',
    }
    for estrutura in estruturas:
      executar_busca(estrutura, chave, **kwargs)

  # Cenário: Pior Caso
  arranjo_ordenado = sorted(arranjo)
  chave  = gerar_chave_pior_caso(arranjo_ordenado)
  lista  = criar_lista(arranjo_ordenado)
  arvore = Arvore.pior_caso(arranjo_ordenado)
  estruturas = (lista, arvore)
  for rodada in range(0, EXEC_PIOR_CASO):
    kwargs = {
      'min': menor,
      'max': maior,
      'tamanho_arranjo': tamanho_arranjo,
      'presente': chave in arranjo_ordenado,
      'rodada': rodada,
      'cenario': 'Pior Caso',
    }
    for estrutura in estruturas:
      elemento, _ = executar_busca(estrutura, chave, **kwargs)
      posicao = elemento.indice if elemento else None
      if posicao != tamanho_arranjo - 1:
        raise IndexError(f'Resultado diferente do esperado - método {estrutura.__class__.__name__}. Esperado {tamanho_arranjo - 1} - Encontrado: {posicao}')


### Calcular as estatíticas

In [20]:
df_bruto = pd.DataFrame(resultados) # Antes de descartar os maiores valores
df = df_bruto.copy()                # Será filtrado para descartar os maiores valores

In [21]:
# Descartar os EXEC_DESCARTAR_MAIOR maiores tempos - devido ao warm up e ao fato do ambiente ser compartilhado
get_tamanho_metodo_cenario = lambda df: df[['tamanho_arranjo', 'metodo', 'cenario']].drop_duplicates().itertuples(index=False)
get_max_tempo_idx = lambda df, tamanho, metodo, cenario: df[(df['cenario'] == cenario) & (df['metodo'] == metodo) & (df['tamanho_arranjo'] == tamanho)]['tempo'].idxmax()

for tamanho, metodo, cenario in get_tamanho_metodo_cenario(df_bruto):
  for _ in range(0, EXEC_DESCARTAR_MAIOR):
    try:
      max_tempo_idx = get_max_tempo_idx(df, tamanho, metodo, cenario)
    except ValueError: # Caso ocorra é seguro ignorar
      pass
    else:
      df.drop(max_tempo_idx, inplace=True)

In [22]:
# Separa os dados de ordenação dos dados de busa
df_busca = df[df['cenario'] != 'Ordenação'].copy()

In [23]:
# Transforma os valores booleanos (True e False) em inteiros (1 e 0, respectivamente)
# Para calcular o % médio de buscas nos quais a chave estava presente no arranjo
# Pois a chave ausente corresponde ao pior caso da busca sequencial, e dependendo do valor, da binária também
df_busca['presente'] = df_busca['presente'].replace([True, False], [int(True), int(False)])

In [24]:
# Calcula as estatísticas descritivas: média, desvio padrão, mínimo, mediana (50% percentil) e máximo
# para as colunas tempo (de busca), número de comparações, % de buscas com chave pertencente ao arranjo
# e profundidade (altura das árvores)
# dos resultados agrupados por tamanho do arranjo e método de busca (sequencial, por saltos e binária)
# serparada para os cenário de busca aleatório ou pior caso
df_aleatorio = df_busca[df_busca['cenario'] == 'Aleatório'].groupby(
    ['tamanho_arranjo', 'metodo']
)[['tempo', 'comparacoes', 'presente', 'profundidade']].describe(percentiles=[])

# não selecionamos a coluna presente no cenário pior caso pois sempre será 1, não agregando informação
df_pior_caso = df_busca[df_busca['cenario'] == 'Pior Caso'].groupby(
    ['tamanho_arranjo', 'metodo']
)[['tempo', 'comparacoes', 'profundidade']].describe(percentiles=[])

### Exibir resultados

In [25]:
# Formatadores
# transforma o float de s para ms e aplica a formação de acordo com a localização
to_ms = lambda s: locale.format_string('%.2f', 1000*s, grouping=True)
# aplica a formação de acordo com a localização - inteiros e floats
locale_format_int = lambda d: '-' if pd.isna(d) else locale.format_string('%d', d, grouping=True)
locale_format_float = lambda f: '-' if pd.isna(f) else locale.format_string('%.2f', f, grouping=True)

In [26]:
def renomear_coluna(coluna: str) -> str:
  if coluna == None:
    return None
  return coluna.\
    replace('tempo','Tempo (ms)').\
    replace('comparacoes','Comp.').\
    replace('presente', 'x ∈ A').\
    replace('profundidade', 'Alt.').\
    replace('mean','x̄').\
    replace('std', 'σ')

In [27]:
# Prepara o DataFrame para exibição/apresentação
def reformatar_df(df: pd.DataFrame, cenario: Union[str,None] = None) -> pd.DataFrame:
  # remover colunas indesejadas
  colunas_selecionadas = [coluna for coluna in df.columns if coluna[1] not in ('count','min','50%', 'max')]
  novo_df = df[colunas_selecionadas].copy()

  # renomear colunas
  if cenario is None:
    colunas_renomeadas = [(renomear_coluna(medida), renomear_coluna(estatistica)) for medida,estatistica in novo_df.columns.values]
  else:   # adiciona o cenário ao nome das colunas - MultiIndex
    colunas_renomeadas = [(cenario, renomear_coluna(medida), renomear_coluna(estatistica)) for medida,estatistica in novo_df.columns.values]
  novo_df.columns = novo_df.columns.values
  novo_df.columns = pd.MultiIndex.from_tuples(colunas_renomeadas)

  # reformatar valores dos índices
  df_idx_names = novo_df.index.names
  df_idx_values = [(locale_format_int(tamanho), metodo) for tamanho,metodo in novo_df.index.values]
  novo_df.index = pd.MultiIndex.from_tuples(df_idx_values, names=df_idx_names)

  # renomear colunas dos índices
  novo_df.rename_axis(
    index={
      'tamanho_arranjo': 'Tam.',
      'metodo': 'Busca' if cenario else 'Ord.'  # Pois na ordenação o cenário não é preenchido
    },
    inplace=True
  )

  return novo_df

In [28]:
from pandas.io.formats.style import Styler
def exibir_df(df: pd.DataFrame) -> Styler:
  # associa os formatadores às colunas
  formatters={
    col:(to_ms if 'Tempo' in col[len(col) - 2] else # Formatador próprio (to_ms) para colunas de tempo
        locale_format_float if len(col)!=3 or (col[0] != 'Pior Caso' and col[1] != 'Alt.') else # demais colunas são float
        locale_format_int)  # Colunas com inteiros: Pior caso (pois não variam terminando em .0), altura e amanho do arranjo
    for col in df.columns
  }

  # Exibe o df aplicando os formatadores
  return df.style.format(formatters)

In [29]:
# remove colunas não informativas
df_estatisticas_busca = reformatar_df(df_aleatorio, 'Aleatório').join(reformatar_df(df_pior_caso, 'Pior Caso')).drop([
    ('Pior Caso', 'Comp.', 'σ'),  # Será sempre 0
    ('Pior Caso', 'Alt.', 'σ'),   # Será sempre 0 - pois a árvore sempre terá a altura == ao tamanho
    ('Aleatório', 'Alt.', 'σ'),   # Será sempre 0 - pois a árvore de busca não muda - apenas as chaves
    ('Aleatório', 'x ∈ A', 'σ'),  # Estamos interessados apenas na média, que nos informa o % de chaves pertencente ao arranjo
], axis=1)

In [30]:
exibir_df(df_estatisticas_busca)

### Salvar resultados

In [31]:
import os # Manipulação de FS

def to_latex(df: pd.DataFrame, hide_index: bool = False, **kwargs) -> str:
  if hide_index:
    new_str = exibir_df(df).hide_index().to_latex(hrules=True, **kwargs)
  else:
    new_str = exibir_df(df).to_latex(hrules=True, **kwargs)
  replacements = {
      'x̄': r'$\bar{x}$',
      'σ': r'$\sigma$',
      'x ∈ A': r'$x \in A$',
  }
  for old, new in replacements.items():
    new_str = new_str.replace(old, new)
  return new_str

def save_text(text: str, path: str):
  with open(path, mode='w') as fout:
    fout.write(text)

if not os.path.exists('./resultados'):
  os.mkdir('./resultados')

# Todas as execuções
df_bruto.to_csv('./resultados/dados-brutos.csv', sep=';')
df.to_csv('./resultados/resultados-todos.csv', sep=';')
df.style.to_excel('./resultados/resultados-todos.xlsx')
# Buscas
df_busca.to_csv('./resultados/resultados-busca.csv', sep=';')
df_busca.style.to_excel('./resultados/resultados-busca.xlsx')

# Gerar tabela latex - semi-formatada para inclusão no artigo
save_text(
  to_latex(
    df_estatisticas_busca,
    clines='skip-last;data',
    caption='Comparação empírica do desempenho dos métodos de busca em Listas e Árvores',
    label='tab:resultados_listas_arvores'
  ),
  os.path.join('./resultados', 'tabela-listas_arvores.tex')
)

In [32]:
import shutil
from google.colab import files

shutil.make_archive('resultados', 'zip', 'resultados')

files.download('resultados.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Informações sobre o ambiente

In [33]:
import platform

# Plataforma
print(platform.platform())
# Versão Python
print(platform.python_implementation() + ': ' + platform.python_version())
# Pandas
print(pd.show_versions())

Linux-5.15.107+-x86_64-with-glibc2.31
CPython: 3.10.12


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



INSTALLED VERSIONS
------------------
commit           : 2e218d10984e9919f0296931d92ea851c6a6faf5
python           : 3.10.12.final.0
python-bits      : 64
OS               : Linux
OS-release       : 5.15.107+
Version          : #1 SMP Sat Apr 29 09:15:28 UTC 2023
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : None
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 1.5.3
numpy            : 1.22.4
pytz             : 2022.7.1
dateutil         : 2.8.2
setuptools       : 67.7.2
pip              : 23.1.2
Cython           : 0.29.35
pytest           : 7.2.2
hypothesis       : None
sphinx           : 3.5.4
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : 4.9.2
html5lib         : 1.1
pymysql          : None
psycopg2         : 2.9.6
jinja2           : 3.1.2
IPython          : 7.34.0
pandas_datareader: 0.10.0
bs4              : 4.11.2
bottleneck       : None
brotli           

In [34]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          12982        1846        7851           1        3284       10849
Swap:             0           0           0


In [35]:
!cat /proc/meminfo

MemTotal:       13294264 kB
MemFree:         8039688 kB
MemAvailable:   11110092 kB
Buffers:           66452 kB
Cached:          3194936 kB
SwapCached:            0 kB
Active:           855624 kB
Inactive:        4187224 kB
Active(anon):       1032 kB
Inactive(anon):  1781744 kB
Active(file):     854592 kB
Inactive(file):  2405480 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              2040 kB
Writeback:             0 kB
AnonPages:       1777636 kB
Mapped:           330432 kB
Shmem:              1316 kB
KReclaimable:     102576 kB
Slab:             137708 kB
SReclaimable:     102576 kB
SUnreclaim:        35132 kB
KernelStack:        4240 kB
PageTables:        15140 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6647132 kB
Committed_AS:    3225972 kB
VmallocTotal:   34359738367 kB
VmallocUsed:        9236 kB
VmallocChunk:          0 kB
Percpu:          

In [36]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

In [37]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 